In [ ]:
from collections import defaultdict
import time
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import mysql.connector
conn = mysql.connector.connect(user='jupiter', password='JupiterGroup$42',
                              host='localhost',
                              database='andromeda')
import MetaTrader5 as mt5
if not mt5.initialize(path="C:/MetaTrader_A/MetaTrader 5/terminal64.exe",login=1049732, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

    
    
## DATABASE TOOLS


def deleteTableParameters():
    mycursor = conn.cursor()
    mycursor.execute("DROP TABLE parameters")

def createTableParameters():
    mycursor = conn.cursor()
    mycursor.execute("CREATE TABLE parameters (f1 VARCHAR(255), f2 FLOAT, f3 FLOAT)")
    
def populateTableParameters(parameterList):
    mycursor = conn.cursor()
    mycursor.executemany("INSERT INTO parameters (f1,f2,f3) VALUES (%s,%s,%s)",parameterList)
    conn.commit()
    
def updateTableParameters(ddx_t_name,ddx_t,ddx_t_interval,ema_t_name,ema_t,ema_t_interval,ema_ddx_t_name,ema_ddx_t,ema_ddx_t_interval,rsi_t_name,rsi_t,rsi_t_interval,cm_name,cm,cm_interval,lev_weeks_name,lev_weeks,lev_weeks_interval,sav_i_name,sav_i,sav_i_interval):
    mycursor = conn.cursor()
    mycursor.execute("DELETE FROM parameters")
    conn.commit()
    parameterList = formParameterList(ddx_t_name,ddx_t,ddx_t_interval,ema_t_name,ema_t,ema_t_interval,ema_ddx_t_name,ema_ddx_t,ema_ddx_t_interval,rsi_t_name,rsi_t,rsi_t_interval,cm_name,cm,cm_interval,lev_weeks_name,lev_weeks,lev_weeks_interval,sav_i_name,sav_i,sav_i_interval)
    populateTableParameters(parameterList)
    
def createTableOriginalParameters():
    mycursor = conn.cursor()
    mycursor.execute("CREATE TABLE originalparameters (f1 VARCHAR(255), f2 FLOAT, f3 FLOAT)")
    
def populateTableOriginalParameters(parameterList):
    mycursor = conn.cursor()
    mycursor.executemany("""INSERT INTO originalparameters (f1,f2,f3) VALUES (%s,%s,%s)""",parameterList)
    conn.commit()
    
def deleteTableResults():
    mycursor = conn.cursor()
    mycursor.execute("DROP TABLE results")

def createTableResults():
    mycursor = conn.cursor()
    mycursor.execute("CREATE TABLE results (combination VARCHAR(255), net FLOAT)")
    
def formParameterList(ddx_t_name,ddx_t,ddx_t_interval,ema_t_name,ema_t,ema_t_interval,ema_ddx_t_name,ema_ddx_t,ema_ddx_t_interval,rsi_t_name,rsi_t,rsi_t_interval,cm_name,cm,cm_interval,lev_weeks_name,lev_weeks,lev_weeks_interval,sav_i_name,sav_i,sav_i_interval):
    parameterList = []
    parameterList.append((ddx_t_name,ddx_t,ddx_t_interval))
    parameterList.append((ema_t_name,ema_t,ema_t_interval))
    parameterList.append((ema_ddx_t_name,ema_ddx_t,ema_ddx_t_interval))
    parameterList.append((rsi_t_name,rsi_t,rsi_t_interval))
    parameterList.append((cm_name,cm,cm_interval))
    parameterList.append((lev_weeks_name,lev_weeks,lev_weeks_interval))
    parameterList.append((sav_i_name,sav_i,sav_i_interval))
    return parameterList

def resetParameters(ddx_t_name,ddx_t_interval,ema_t_name,ema_t_interval,ema_ddx_t_name,ema_ddx_t_interval,rsi_t_name,rsi_t_interval,cm_name,cm_interval,lev_weeks_name,lev_weeks_interval,sav_i_name,sav_i_interval):
    ddx_t, ema_t, ema_ddx_t, rsi_t, cm, lev_weeks, sav_i = getOriginalParameters()
    updateTableParameters(ddx_t_name,ddx_t,ddx_t_interval,ema_t_name,ema_t,ema_t_interval,ema_ddx_t_name,ema_ddx_t,ema_ddx_t_interval,rsi_t_name,rsi_t,rsi_t_interval,cm_name,cm,cm_interval,lev_weeks_name,lev_weeks,lev_weeks_interval,sav_i_name,sav_i,sav_i_interval)

def getParameters():
    mycursor = conn.cursor()
    mycursor.execute("SELECT * FROM parameters")
    result = mycursor.fetchall()
    ddx_t = int(result[0][1])
    ema_t = int(result[1][1])
    ema_ddx_t = int(result[2][1])
    rsi_t = int(result[3][1])
    cm = float(result[4][1])
    lev_weeks = int(result[5][1])
    sav_i = int(result[6][1])
    return ddx_t, ema_t, ema_ddx_t, rsi_t, cm, lev_weeks, sav_i

def getOriginalParameters():
    mycursor = conn.cursor()
    mycursor.execute("SELECT * FROM originalparameters")
    result = mycursor.fetchall()
    ddx_t = int(result[0][1])
    ema_t = int(result[1][1])
    ema_ddx_t = int(result[2][1])
    rsi_t = int(result[3][1])
    cm = float(result[4][1])
    lev_weeks = int(result[5][1])
    sav_i = int(result[6][1])
    return ddx_t, ema_t, ema_ddx_t, rsi_t, cm, lev_weeks, sav_i

    
    
## OPERATIONAL TOOLS


def MetaLink_range(pair,chart,from_date,to_date):
    if chart == "D1":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_D1, from_date, to_date)
    elif chart == "H8":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_H8, from_date, to_date)
    elif chart == "H4":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_H4, from_date, to_date)
    elif chart == "H3":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_H3, from_date, to_date)
    elif chart == "H2":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_H2, from_date, to_date)
    elif chart == "H1":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_H1, from_date, to_date)
    elif chart == "M30":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_M30, from_date, to_date)
    elif chart == "M15":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_M15, from_date, to_date)
    elif chart == "M5":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_M5, from_date, to_date)
    elif chart == "M1":
        coreFeed = mt5.copy_rates_range(pair, mt5.TIMEFRAME_M1, from_date, to_date)
    return coreFeed

def MetaLink_pos(pair, chart, candles):
    if chart == "D1":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_D1, 0, candles)
    elif chart == "H8":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H8, 0, candles)
    elif chart == "H4":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H4, 0, candles)
    elif chart == "H3":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H3, 0, candles)
    elif chart == "H2":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H2, 0, candles)
    elif chart == "H1":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H1, 0, candles)
    elif chart == "M30":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_M30, 0, candles)
    elif chart == "M15":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_M15, 0, candles)
    elif chart == "M5":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_M5, 0, candles)
    elif chart == "M1":
        coreFeed = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_M1, 0, candles)
    return coreFeed

def Pip_Screen(pair,data):
    base, quote = pair[:3], pair[3:]
    if quote == "JPY":
        data = data*100
    elif quote == 'NOK' or quote == 'SEK' or quote == 'DKK' or quote == 'MXN':
        data = data*1000
    else:
        data = data*10000
    return data

def levelgrab(pairsList, chart, lev_from_date, lev_to_date, cm, sav_i):
    levslist = []
    for pair in pairsList:
        coreFeed = MetaLink_range(pair, chart, lev_from_date, lev_to_date)
        SCMavg = np.average(abs(coreFeed['close'] - coreFeed['open']))
        SCMstd = np.std(abs(coreFeed['close'] - coreFeed['open']))
        SCM = SCMavg + SCMstd*cm
        y = savgol_filter(coreFeed['close'], sav_i, 3)
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)
        levs = list(levs)
        mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
        while mindist < SCM:
            i = 0
            h = -1
            j = 1
            for c in range(0, len(levs)):
                if j == len(levs):
                    j = -1
                if abs(levs[i]-levs[h]) < SCM:
                    levs[i] = 0
                elif abs(levs[i]-levs[j]) < SCM:
                    levs[i] = levs[i] - abs(levs[i]-levs[j])/2
                i += 1
                h += 1
                j += 1
            levs = [v for v in levs if v != 0]
            mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])
        levs = tuple(levs)
        levslist.append(levs)
    return levslist



## TECHNICAL ANALYSIS TOOLBOX


def ddx(pair, feed, ddx_t):
    feedPips = Pip_Screen(pair, feed['close'])
    n = len(feedPips) - ddx_t - 1
    slope = feedPips[-1] - feedPips[n]
    return slope
    
def ema(pair, feed, ema_t, ema_ddx_t):
    maList = []
    nn = len(feed['close']) - ema_t - (ema_ddx_t-1) - 1
    feed = feed[nn:]
    for g in range(ema_ddx_t,0,-1):
        n = ema_t - (g-1)
        if g == 1:
            newFeed = feed[n:]
        else:
            newFeed = feed[n:-1*(g-1)]
        highPips = Pip_Screen(pair, newFeed['high'])
        lowPips = Pip_Screen(pair, newFeed['low'])
        emaHigh = np.max(highPips)
        emaLow = np.min(lowPips)
        movingAverage = Pip_Screen(pair, newFeed['close'][-1]) - (emaHigh + emaLow)/2
        maList.append(movingAverage)
        n += 1
    maSlope = maList[-1] - maList[0]
    return movingAverage, maSlope
    
def rsi(pair, feed, rsi_t):
    n = len(feed['close']) - rsi_t - 1
    C = Pip_Screen(pair,feed['close'][n:])
    O = Pip_Screen(pair,feed['open'][n:])
    T = (C - O)/O
    ups = np.average(T[T>0])
    downs = abs(np.average(T[T<0]))
    N = (C[-1] - O[-1])/O[-1]
    if N > 0:
        gain = N
        loss = 0
    else:
        gain = 0
        loss = N
    strengthAverage = 100 - (100/(1+((ups*13+gain)/(downs*13+loss))))
    return strengthAverage

def sr(feed, levels, cm):
    currentPrice = feed['close'][-1]
    lastone = abs(levels[0] - currentPrice)
    q = 0
    for x in levels:
        if q == 0:
            Q = q
        leveldiff = abs(x - currentPrice)
        if leveldiff < lastone:
            Q = q
            CM_check = leveldiff
        lastone = leveldiff
        q += 1
    primaryLevel = levels[Q]
    distPrimaryLevel = currentPrice - primaryLevel
    try:
        nextUpLevel = levels[Q-1]   # Needs work
    except IndexError:
        nextUpLevel = 0
    try:
        nextDownLevel = levels[Q+1] # Needs work
    except IndexError:
        nextDownLevel = 0
    return primaryLevel, distPrimaryLevel, nextUpLevel, nextDownLevel
    
    
    
## PARAMETER PLAYBOOK & HIERARCHY



    


## PROCEDURE


def setup():
    pairsList = []
    symbols=mt5.symbols_get()
    for s in symbols:
        if "TRY" not in s.name:
            if "Crude" not in s.name:
                pairsList.append(s.name)
    pairsList = pairsList[:-12]
    levslist = levelgrab(pairsList, chart, lev_from_date, lev_to_date, cm, sav_i)
    return pairsList, levslist

def tradeCrunch(subREADINGS, subLONGS, subSHORTS, coreFeed):
    subCrunchedLONGS = []
    subCrunchedSHORTS = []
    p = 0
    for long in subLONGS:
        point = list(coreFeed['time']).index(long[1])
        crunchFeed = Pip_Screen(long[0],coreFeed['close'][point:])
        entry = crunchFeed[0]
        takeprofit = long[2]
        stoploss = long[3]
        for row in crunchFeed:
            if takeprofit == 0:
                takeprofit = entry + (entry-stoploss)
            if stoploss == 0:
                stoploss = entry - (takeprofit-entry)
            if row > takeprofit:
                net = takeprofit - entry
                break
            elif row < stoploss:
                net = stoploss - entry
                break
        subCrunchedLONGS.append((subREADINGS[p],net))
        p += 1
    p = 0
    for short in subSHORTS:
        point = list(coreFeed['time']).index(short[1])
        crunchFeed = Pip_Screen(short[0],coreFeed['close'][point:])
        entry = crunchFeed[0]
        takeprofit = short[2]
        stoploss = short[3]
        for row in crunchFeed:
            if takeprofit == 0:
                takeprofit = entry - (stoploss-entry)
            if stoploss == 0:
                stoploss = entry + (entry-takeprofit)
            if row < takeprofit:
                net = entry- takeprofit
                break
            elif row > stoploss:
                net = entry - stoploss
                break
        subCrunchedSHORTS.append((subREADINGS[p],net))
        p += 1
    return subCrunchedLONGS, subCrunchedSHORTS
    
def sequence(riskusd, pair, chart, core_weeks, from_date, to_date, lev_from_date, lev_to_date, levels, ddx_t, ema_t, ema_ddx_t, rsi_t, cm):
    subLONGS = []
    subSHORTS = []
    subREADINGS = []
    coreFeed = MetaLink_range(pair, chart, from_date, to_date)
    pin = max(ddx_t, ema_t, ema_t+ema_ddx_t, rsi_t)
    i = 0
    for f in range(len(coreFeed['close'])):
        if f > pin:
            feed = coreFeed[i:f]
            slope = ddx(pair, feed, ddx_t)
            movingAverage, maSlope = ema(pair, feed, ema_t, ema_ddx_t)
            if i == 0:
                slope2 = 0
                maSlope2 = 0
            else:
                slope2 = slope - prevSlope
                maSlope2 = maSlope - prevMaSlope
            strengthAverage = rsi(pair, feed, rsi_t)
            primaryLevel, distPrimaryLevel, nextUpLevel, nextDownLevel = sr(feed, levels, cm)
            timeStamp = feed['time'][-1]
            subLONGS.append((pair,timeStamp,nextUpLevel,nextDownLevel))
            subSHORTS.append((pair,timeStamp,nextDownLevel,nextUpLevel))
            subREADINGS.append((slope,slope2,movingAverage,strengthAverage))
            prevSlope = slope
            prevMaSlope = maSlope
            i += 1
    return subREADINGS, subLONGS, subSHORTS, coreFeed
    
def core(riskusd, pairsList, chart, from_date, to_date, lev_from_date, lev_to_date, ddx_t, ema_t, ema_ddx_t, rsi_t, cm):
    LONGS = []
    SHORTS = []
    ia = 0
    for pair in pairsList:
        levels = levslist[ia]
        subREADINGS, subLONGS, subSHORTS, coreFeed = sequence(riskusd, pair, chart, core_weeks, from_date, to_date, lev_from_date, lev_to_date, levels, ddx_t, ema_t, ema_ddx_t, rsi_t, cm)
        subCrunchedLONGS, subCrunchedSHORTS = tradeCrunch(subREADINGS, subLONGS, subSHORTS, coreFeed)
        LONGS.append(subCrunchedLONGS)
        SHORTS.append(subCrunchedSHORTS)
        ia += 1
    return LONGS, SHORTS
    
    

################################################################################
    
    
## CONTROL PANEL


core_weeks = 10

timezone = pytz.timezone("Etc/UTC")
to_date = datetime(2020, 9, 1, hour=0, tzinfo=timezone)
delta1=timedelta(
            days=0,
            seconds=0,
            microseconds=0,
            milliseconds=0,
            minutes=0,
            hours=0,
            weeks=core_weeks
        )
from_date = to_date - delta1

riskusd = 50.00   

chart = "H1"

ddx_t_name = "ddx_t"
ema_t_name = "ema_t"
ema_ddx_t_name = "ema_ddx_t"
rsi_t_name = "rsi_t"
cm_name = "cm"
lev_weeks_name = "lev_weeks"
sav_i_name = "sav_i"

#ddx_t = 4
#ema_t = 4
#ema_ddx_t = 4
#rsi_t = 4
#cm = 0
#lev_weeks = 25
#sav_i = 9

ddx_t_interval = 2
ema_t_interval = 2
ema_ddx_t_interval = 2
rsi_t_interval = 1
cm_interval = 0.5
lev_weeks_interval = 5
sav_i_interval = 2

#parameterList = formParameterList(ddx_t_name,ddx_t,ddx_t_interval,ema_t_name,ema_t,ema_t_interval,ema_ddx_t_name,ema_ddx_t,ema_ddx_t_interval,rsi_t_name,rsi_t,rsi_t_interval,cm_name,cm,cm_interval,lev_weeks_name,lev_weeks,lev_weeks_interval,sav_i_name,sav_i,sav_i_interval)
#deleteTableParameters()
#createTableParameters()
#populateTableParameters(parameterList)
#createTableOriginalParameters()
#populateTableOriginalParameters(parameterList)

## ddx_t,ema_t,rsi_t,cm,lev_weeks,sav_i

## Iterations of the entire module
iterMaximum = 150

breaker = 1



################################################################################


## MOTHERBOARD

#resetParameters(ddx_t_name,ddx_t_interval,ema_t_name,ema_t_interval,ema_ddx_t_name,ema_ddx_t_interval,rsi_t_name,rsi_t_interval,cm_name,cm_interval,lev_weeks_name,lev_weeks_interval,sav_i_name,sav_i_interval)
lastChange = 0
yes = ["yes","Yes","Y","y","1","ye","Ye"]
iterCount = 1
R = 0
proceed = input("\n\nYou are about to activate ANDROMEDA.\n\nAre you sure you "+
                "would like to proceed?\n\n")
if proceed in yes:
    print("\n\nLaunching ANDROMEDA...\n\n")
    while iterCount <= iterMaximum:
        startTime = datetime.now()
        ddx_t, ema_t, ema_ddx_t, rsi_t, cm, lev_weeks, sav_i = getParameters()
        #print(ddx_t,ema_t,ema_ddx_t,rsi_t,cm,lev_weeks,sav_i)
        lev_to_date = to_date
        delta2=timedelta(
            days=0,
            seconds=0,
            microseconds=0,
            milliseconds=0,
            minutes=0,
            hours=0,
            weeks=lev_weeks
        )
        lev_from_date = lev_to_date - delta2
        pairsList, levslist = setup()
        LONGS, SHORTS = core(riskusd, pairsList, chart, from_date, to_date, lev_from_date, lev_to_date, ddx_t, ema_t, ema_ddx_t, rsi_t, cm)
        #print(LONGS)
        #print(SHORTS)
        test = 1 ## Simulates the current output being more/less profit than the last
        if test == 1:
            lastChange = iterCount
            if R == 0:
                ddx_t = ddx_t + ddx_t_interval
            elif R == 1:
                ema_t = ema_t + ema_t_interval
            elif R == 2:
                rsi_t = rsi_t + rsi_t_interval
            elif R == 3:
                cm = cm + cm_interval
            elif R == 4:
                lev_weeks = lev_weeks + lev_weeks_interval
            elif R == 5:
                sav_i = sav_i + sav_i_interval
        if R == 5:
            R = -1
        R += 1
        updateTableParameters(ddx_t_name,ddx_t,ddx_t_interval,ema_t_name,ema_t,ema_t_interval,ema_ddx_t_name,ema_ddx_t,ema_ddx_t_interval,rsi_t_name,rsi_t,rsi_t_interval,cm_name,cm,cm_interval,lev_weeks_name,lev_weeks,lev_weeks_interval,sav_i_name,sav_i,sav_i_interval)
        endTime = datetime.now()
        elapsed = endTime - startTime
        #print(R," ",ddx_t,ema_t,ema_ddx_t,rsi_t,cm,lev_weeks,sav_i)
        print("Elapsed Time for Iteration #" + str(iterCount) + " :  " + str(elapsed))
        if iterCount - lastChange >= breaker:
            break
        iterCount += 1
    print("Optimization Complete.")
else:
    print("ANDROMEDA aborted.")
resetParameters(ddx_t_name,ddx_t_interval,ema_t_name,ema_t_interval,ema_ddx_t_name,ema_ddx_t_interval,rsi_t_name,rsi_t_interval,cm_name,cm_interval,lev_weeks_name,lev_weeks_interval,sav_i_name,sav_i_interval)



You are about to activate ANDROMEDA.

Are you sure you would like to proceed?

y


Launching ANDROMEDA...




C:\Users\User\.conda\envs\EnvA\lib\site-packages\numpy\lib\function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\User\.conda\envs\EnvA\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Elapsed Time for Iteration #1 :  0:02:36.242629
Elapsed Time for Iteration #2 :  0:02:32.378008
Elapsed Time for Iteration #3 :  0:02:31.716837
Elapsed Time for Iteration #4 :  0:02:32.088835
Elapsed Time for Iteration #5 :  0:02:31.866775
Elapsed Time for Iteration #6 :  0:03:21.647543
Elapsed Time for Iteration #7 :  0:02:46.482750
Elapsed Time for Iteration #8 :  0:02:46.049296
Elapsed Time for Iteration #9 :  0:02:45.977717
Elapsed Time for Iteration #10 :  0:02:46.062670
